In [8]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Customer metadata: PV capacity (kWp), EV ownership (True/False), and feeder group
customers = {
    'H1': {'Transformer': 1},
    'H2': {'Transformer': 2},
    'H3': {'Transformer': 2},
    'H4': {'Transformer': 3},
    'H5': {'Transformer': 3},
    'H6': {'Transformer': 3},
    'H7': {'Transformer': 4},
    'H8': {'Transformer': 4},
    'H9': {'Transformer': 1},
    'H10': {'Transformer': 1},
    'H11': {'Transformer': 1},
    'H12': {'Transformer':3},
    'H13': {'Transformer': 1},
    'H14': {'Transformer': 1},
    'H15': {'Transformer': 4},
}

customer_ids = list(customers.keys())

# Time range: 15-min intervals from Mar 4, 2024, 09:15 to Feb 19, 2025, 07:30
dates = pd.date_range(
    start='2020-01-01 01:01:00',
    end='2020-12-11 23:59:00',    #'2020-12-11 23:59:00'
    freq='1min'
).strftime('%Y-%m-%d %H:%M:%S')


In [3]:
# Load consumption (EBez) and generation (EAbg) data from CSV files
def load_data(customer_ids):
    data = {}
    for cid in customer_ids:
        df_file = f"{cid}_Wh.csv"

        # Check if files exist
        if not os.path.exists(df_file) or not os.path.exists(df_file):
            raise FileNotFoundError(f"Missing files for customer {cid}")

        # Read CSVs and remove duplicate timestamps (keep last)
        df = pd.read_csv(df_file)
        
        # Convert 'date' column to datetime
        df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d %H:%M:%S", errors="coerce")

        start='2020-01-01 01:01:00'
        end='2020-12-11 23:59:00'
        # Filter rows based on date range
        df = df[(df["date"] >= start) & (df["date"] <= end)]


        df[' Production(Wh)'] = pd.to_numeric(df[' Production(Wh)'], errors='coerce')
        df[' Consumption(Wh)'] = pd.to_numeric(df[' Consumption(Wh)'], errors='coerce')
        df[' Production(KWh)'] = df[' Production(Wh)']/1000
        df[' Consumption(KWh)'] = df[' Consumption(Wh)']/1000

        # # Set datetime as index
        # df.set_index("date", inplace=True)

        data[cid] = {
            'EBez': df.set_index("date")[' Consumption(KWh)'].rename_axis(None),
            'EAbg': df.set_index("date")[' Production(KWh)'].rename_axis(None)
        }
        
    return data

# Load data for all customers
data = load_data(customer_ids)

In [4]:
feeder_ids = set(customers[cid]['Transformer'] for cid in customer_ids)
total_consumption = {cid:0 for cid in customer_ids}
total_consumption_feeder = {feeder:0 for feeder in feeder_ids}

for timestamp in dates:
    for cid in customer_ids:
        total_consumption[cid] += max(0, data[cid]['EBez'].get(timestamp, 0) - data[cid]['EAbg'].get(timestamp, 0))
        total_consumption_feeder[customers[cid]['Transformer']] += max(0, data[cid]['EBez'].get(timestamp, 0) - data[cid]['EAbg'].get(timestamp, 0))

static_weight_global = {cid: 0 for cid in customer_ids}
static_weight_feeder = {cid:0 for cid in customer_ids}
for cid in customer_ids:
    static_weight_global[cid] = total_consumption[cid]/ sum(total_consumption.values())
    static_weight_feeder[cid] = total_consumption[cid]/ total_consumption_feeder[customers[cid]['Transformer']]

ranked_cids = sorted(customer_ids, key=lambda x:total_consumption[x])


In [9]:
# Allocate PV surplus energy based on specified method
def static_energy_allocation(data, customers, method, timestamp, dk):
    # Initialize dictionaries for consumption, generation, and allocation, surplus and energy need after allocation, ...
    consumption = {cid: data[cid]['EBez'].get(timestamp, 0) for cid in customer_ids}
    pv_generation = {cid: data[cid]['EAbg'].get(timestamp, 0) for cid in customer_ids}
    allocations = {cid: 0.0 for cid in customer_ids}
    glob_allocation = {cid: 0.0 for cid in customer_ids}
    surplus = {cid: max(0, pv_generation[cid] - consumption[cid]) for cid in customer_ids}
    sold_in_community = {cid: 0.0 for cid in customer_ids}
    need = {cid: max(0, consumption[cid] - pv_generation[cid]) for cid in customer_ids}
    same_feeder = {cid: 0.0 for cid in customer_ids}  # Energy from same feeder
    other_feeder = {cid: 0.0 for cid in customer_ids}  # Energy from other feeders
    static_weight_global = {'H1': 0.035996166405492025,
                    'H2': 0.05623288902937398,
                    'H3': 0.06264794797788295,
                    'H4': 0.09426005290328053,
                    'H5': 0.022102153559965927,
                    'H6': 0.0898110720704961,
                    'H7': 0.08128318845201273,
                    'H8': 0.0975357222725164,
                    'H9': 0.05433129086844193,
                    'H10': 0.09761165578877164,
                    'H11': 0.07263384123665334,
                    'H12': 0.04189870158506696,
                    'H13': 0.029688845186062664,
                    'H14': 0.08055554323311805,
                    'H15': 0.08341092943086492}
    static_weight_feeder = {'H1': 0.09707249974252914,
                    'H2': 0.4730189528021227,
                    'H3': 0.5269810471985624,
                    'H4': 0.3799705749042327,
                    'H5': 0.08909572757633796,
                    'H6': 0.3620363413375955,
                    'H7': 0.3099692559927797,
                    'H8': 0.3719474572946784,
                    'H9': 0.14651766411529388,
                    'H10': 0.2632337934180039,
                    'H11': 0.19587498444468202,
                    'H12': 0.16889735618284066,
                    'H13': 0.0800632596320131,
                    'H14': 0.21723779864966597,
 'H15': 0.31808328671328384}
    total_ranked_cids = ['H5',
                    'H13',
                    'H1',
                    'H12',
                    'H9',
                    'H2',
                    'H3',
                    'H11',
                    'H14',
                    'H7',
                    'H15',
                    'H6',
                    'H4',
                    'H8',
                    'H10']
        
    if dk == 'equal':
        if method == 'feeder_aware':
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate within same feeder
            for transformer in set(customers[cid]['Transformer'] for cid in customer_ids):
                t_cids = [cid for cid in customer_ids if customers[cid]['Transformer'] == transformer]
                t_numbers = len(t_cids)
                total_surplus = sum(surplus[cid] for cid in t_cids)
                total_need = sum(need[cid] for cid in t_cids)
                alloc = total_surplus
                total_alloc = 0
                if t_numbers > 0:
                    share = alloc / t_numbers
                if total_surplus > 0 and total_need > 0:
                    for cid in t_cids:
                        if need[cid] > 0:
                            allocations[cid] += min(need[cid], share)
                            total_alloc += min(need[cid], share)
                            same_feeder[cid] += min(need[cid], share)
                            need[cid] -= min(need[cid], share)
                            
                    for cid in t_cids:
                        if surplus[cid] > 0:
                            surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                remaining_pv -= total_alloc

            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder


            # Map customers to their feeders
            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Allocate across all feeders
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            number = len(customer_ids)
            total_alloc = 0
            if number > 0:
                share = alloc / number

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        allocations[cid] += min(need[cid], share)
                        glob_allocation[cid] +=  min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                        
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            # Step 3: Determine source of allocated energy (new logic)
            # Calculate surplus and allocated energy per feeder
            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += glob_allocation[cid] # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = glob_allocation[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder

        elif method == 'feeder_agnostic':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)
            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder
            # Allocate surplus globally
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            number = len(customer_ids)
            total_alloc = 0
            if number > 0:
                share = alloc / number

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        allocations[cid] += min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                        
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += allocations[cid]  # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = allocations[cid] # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = feeder_sold[fid] / feeder_allocated[fid]
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder
        
    elif dk == 'proportional':
        if method == 'feeder_aware':
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate within same feeder
            for transformer in set(customers[cid]['Transformer'] for cid in customer_ids):
                t_cids = [cid for cid in customer_ids if customers[cid]['Transformer'] == transformer]
                total_surplus = sum(surplus[cid] for cid in t_cids)
                total_need = sum(need[cid] for cid in t_cids)
                alloc = total_surplus
                total_alloc = 0

                if total_surplus > 0 and total_need > 0:
                    for cid in t_cids:
                        if need[cid] > 0:
                            share = alloc * static_weight_feeder[cid]
                            allocations[cid] += min(need[cid], share)
                            same_feeder[cid] += min(need[cid], share)
                            total_alloc += min(need[cid], share)
                            need[cid] -= min(need[cid], share)
                    for cid in t_cids:
                        if surplus[cid] > 0:
                            surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                remaining_pv -= alloc

            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder
            
            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())

            # Allocate across all feeders
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            total_alloc = 0

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        share = alloc * static_weight_global[cid]
                        allocations[cid] += min(need[cid], share)
                        glob_allocation[cid] = min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            # Step 3: Determine source of allocated energy (new logic)
            # Calculate surplus and allocated energy per feeder
            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += glob_allocation[cid] # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = glob_allocation[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder

        elif method == 'feeder_agnostic':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate surplus globally
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            total_alloc = 0

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        share = alloc * static_weight_global[cid]
                        allocations[cid] += min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)


            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += allocations[cid]  # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = allocations[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = feeder_sold[fid] / feeder_allocated[fid]
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder
        
    elif dk == 'ranking':
        if method == 'feeder_aware':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate within same feeder
            for transformer in set(customers[cid]['Transformer'] for cid in customer_ids):
                t_cids = [cid for cid in total_ranked_cids if customers[cid]['Transformer'] == transformer]
                total_surplus = sum(surplus[cid] for cid in t_cids)
                total_need = sum(need[cid] for cid in t_cids)
                total_alloc=0
                available_surplus = total_surplus

                if total_surplus > 0 and total_need > 0:
                    for cid in t_cids:
                        if need[cid] > 0:
                            share = min(need[cid], available_surplus)
                            allocations[cid] += share
                            same_feeder[cid] += share
                            total_alloc += share
                            need[cid] -= share 
                            available_surplus -= share
                            if available_surplus <=0:
                                break
                    for cid in t_cids:
                        if surplus[cid] > 0:
                            surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                remaining_pv -= total_alloc

            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate across all feeders
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            ranked_cids = total_ranked_cids
            total_alloc = 0
            available_surplus = total_surplus

            if total_surplus > 0 and total_need > 0:
                for cid in ranked_cids:
                    if need[cid] > 0:
                        share = min(need[cid], available_surplus)
                        allocations[cid] += share
                        glob_allocation[cid] += share
                        total_alloc += share
                        need[cid] -= share 
                        available_surplus -= share
                        if available_surplus <=0:
                            break
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += glob_allocation[cid]  # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = glob_allocation[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder

        elif method == 'feeder_agnostic':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate surplus globally
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            ranked_cids = total_ranked_cids
            total_alloc = 0
            available_surplus = total_surplus

            if total_surplus > 0 and total_need > 0:
                for cid in ranked_cids:
                    if need[cid] > 0:
                        share = min(need[cid], available_surplus)
                        allocations[cid] += share
                        total_alloc += share
                        need[cid] -= share 
                        available_surplus -= share
                        if available_surplus <=0:
                            break
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)


            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += allocations[cid]  # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = allocations[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid]:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder


In [10]:
# Allocate PV surplus energy based on specified method
def dynamic_enegy_allocation(data, customers, method, timestamp, dk):
    # Initialize dictionaries for consumption, generation, and allocation, surplus and energy need after allocation, ...
    consumption = {cid: data[cid]['EBez'].get(timestamp, 0) for cid in customer_ids}
    pv_generation = {cid: data[cid]['EAbg'].get(timestamp, 0) for cid in customer_ids}
    allocations = {cid: 0.0 for cid in customer_ids}
    glob_allocation = {cid: 0.0 for cid in customer_ids}
    surplus = {cid: max(0, pv_generation[cid] - consumption[cid]) for cid in customer_ids}
    sold_in_community = {cid: 0.0 for cid in customer_ids}
    need = {cid: max(0, consumption[cid] - pv_generation[cid]) for cid in customer_ids}
    same_feeder = {cid: 0.0 for cid in customer_ids}  # Energy from same feeder
    other_feeder = {cid: 0.0 for cid in customer_ids}  # Energy from other feeders
    
    if dk == 'equal':
        if method == 'feeder_aware':

            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # # Allocate within same feeder
            for transformer in set(customers[cid]['Transformer'] for cid in customer_ids):
                t_cids = [cid for cid in customer_ids if customers[cid]['Transformer'] == transformer]
                t_numbers = len([cid for cid in t_cids if need[cid] > 0])
                total_surplus = sum(surplus[cid] for cid in t_cids)
                total_need = sum(need[cid] for cid in t_cids)
                alloc = total_surplus
                total_alloc = 0
                if t_numbers > 0:
                    share = alloc / t_numbers
                if total_surplus > 0 and total_need > 0:
                    for cid in t_cids:
                        if need[cid] > 0:
                            allocations[cid] += min(need[cid], share)
                            total_alloc += min(need[cid], share)
                            same_feeder[cid] += min(need[cid], share)
                            need[cid] -= min(need[cid], share)
                        
                    for cid in t_cids:
                        if surplus[cid] > 0:
                            surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                remaining_pv -= total_alloc
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Map customers to their feeders
            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Allocate across all feeders
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            number = len([cid for cid in customer_ids if need[cid] > 0])
            total_alloc = 0
            if number > 0:
                share = alloc / number

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        allocations[cid] += min(need[cid], share)
                        glob_allocation[cid] +=  min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                        
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                    
            # Step 3: Determine source of allocated energy (new logic)
            # Calculate surplus and allocated energy per feeder
            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += glob_allocation[cid] # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = glob_allocation[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)


            return allocations, need, surplus, same_feeder, other_feeder

        elif method == 'feeder_agnostic':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder
            # Allocate surplus globally
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            number = len([cid for cid in customer_ids if need[cid] > 0])
            total_alloc = 0
            if number > 0:
                share = alloc / number

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        allocations[cid] += min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                        
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += allocations[cid]  # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = allocations[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = feeder_sold[fid] / feeder_allocated[fid]
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)


            return allocations, need, surplus, same_feeder, other_feeder
        
    elif dk == 'proportional':
        if method == 'feeder_aware':
            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate within same feeder
            for transformer in set(customers[cid]['Transformer'] for cid in customer_ids):
                t_cids = [cid for cid in customer_ids if customers[cid]['Transformer'] == transformer]
                total_surplus = sum(surplus[cid] for cid in t_cids)
                total_need = sum(need[cid] for cid in t_cids)
                alloc = total_surplus
                total_alloc = 0

                if total_surplus > 0 and total_need > 0:
                    for cid in t_cids:
                        if need[cid] > 0:
                            share = alloc * (need[cid] / total_need)
                            allocations[cid] += min(need[cid], share)
                            total_alloc += min(need[cid], share)
                            same_feeder[cid] += min(need[cid], share)
                            need[cid] -= min(need[cid], share)
                    for cid in t_cids:
                        if surplus[cid] > 0:
                            surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                remaining_pv -= total_alloc

            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder
            
            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Allocate across all feeders
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            total_alloc = 0

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        share = alloc * (need[cid] / total_need)
                        allocations[cid] += min(need[cid], share)
                        glob_allocation[cid] = min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)


            # Step 3: Determine source of allocated energy (new logic)
            # Calculate surplus and allocated energy per feeder
            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += glob_allocation[cid] # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = glob_allocation[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder

        elif method == 'feeder_agnostic':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate surplus globally
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            alloc = total_surplus
            total_alloc = 0

            if total_surplus > 0 and total_need > 0:
                for cid in customer_ids:
                    if need[cid] > 0:
                        share = alloc * (need[cid] / total_need)
                        allocations[cid] += min(need[cid], share)
                        total_alloc += min(need[cid], share)
                        need[cid] -= min(need[cid], share)
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += allocations[cid] # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = allocations[cid]   # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = feeder_sold[fid] / feeder_allocated[fid]
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)
            return allocations, need, surplus, same_feeder, other_feeder
        
    elif dk == 'ranking':
        if method == 'feeder_aware':
            
            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder


            # Allocate within same feeder
            for transformer in set(customers[cid]['Transformer'] for cid in customer_ids):
                t_cids = [cid for cid in customer_ids if customers[cid]['Transformer'] == transformer]
                total_surplus = sum(surplus[cid] for cid in t_cids)
                total_need = sum(need[cid] for cid in t_cids)
                ranked_cids = sorted(t_cids, key=lambda x:need[x])
                total_alloc=0
                available_surplus = total_surplus

                if total_surplus > 0 and total_need > 0:
                    for cid in ranked_cids:
                        if need[cid] > 0:
                            share = min(need[cid], available_surplus)
                            allocations[cid] += share
                            same_feeder[cid] += share
                            total_alloc += share
                            need[cid] -= share 
                            available_surplus -= share
                            if available_surplus <=0:
                                break
                    for cid in t_cids:
                        if surplus[cid] > 0:
                            surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)
                    remaining_pv -= total_alloc
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # # Allocate across all feeders
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            ranked_cids = sorted(customer_ids, key=lambda x:need[x])
            total_alloc = 0
            available_surplus = total_surplus

            if total_surplus > 0 and total_need > 0:
                for cid in ranked_cids:
                    if need[cid] > 0:
                        share = min(need[cid], available_surplus)
                        allocations[cid] += share
                        glob_allocation[cid] += share
                        total_alloc += share
                        need[cid] -= share 
                        available_surplus -= share
                        if available_surplus <=0:
                            break
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += glob_allocation[cid]  # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = glob_allocation[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid] and feeder_allocated[fid] > 0:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)


            return allocations, need, surplus, same_feeder, other_feeder

        elif method == 'feeder_agnostic':

            customer_to_feeder = {cid: customers[cid]['Transformer'] for cid in customer_ids}
            feeder_ids = set(customer_to_feeder.values())
            # Self-consumption
            for cid in customer_ids:
                self_cons = min(pv_generation[cid], consumption[cid])
                surplus[cid] = max(0, pv_generation[cid] - self_cons)
                need[cid] = max(0, consumption[cid] - self_cons)

            remaining_pv = sum(surplus.values())
            if remaining_pv <= 0:
                return allocations, need, surplus, same_feeder, other_feeder

            # Allocate surplus globally
            total_surplus = sum(surplus.values())
            total_need = sum(need.values())
            ranked_cids = sorted(customer_ids, key=lambda x:need[x])
            total_alloc = 0
            available_surplus = total_surplus

            if total_surplus > 0 and total_need > 0:
                for cid in ranked_cids:
                    if need[cid] > 0:
                        share = min(need[cid], available_surplus)
                        allocations[cid] += share
                        total_alloc += share
                        need[cid] -= share 
                        available_surplus -= share
                        if available_surplus <=0:
                            break
                for cid in customer_ids:
                    if surplus[cid] > 0:
                        sold_in_community[cid] = total_alloc * (surplus[cid] / total_surplus)
                        surplus[cid] -= total_alloc * (surplus[cid] / total_surplus)

            feeder_surplus = {fid: 0.0 for fid in feeder_ids}
            feeder_allocated = {fid: 0.0 for fid in feeder_ids}
            feeder_sold = {fid: 0.0 for fid in feeder_ids}

            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                feeder_surplus[fid] += surplus[cid] + sold_in_community[cid]  # Initial surplus before sharing
                feeder_sold[fid] += sold_in_community[cid]  # Energy sold in community
                feeder_allocated[fid] += allocations[cid] # Allocated after self-consumption

            # Step 4: Assign energy source (same_feeder vs. other_feeder)
            for cid in customer_ids:
                fid = customer_to_feeder[cid]
                allocated_after_self = allocations[cid]  # Energy allocated after self-consumption
                if allocated_after_self > 0:
                    if feeder_sold[fid] >= feeder_allocated[fid]:
                        # All allocated energy comes from the same feeder
                        same_feeder[cid] += allocated_after_self
                        other_feeder[cid] += 0.0
                    else:
                        # Calculate proportion from same feeder based on allocated/generated ratio
                        same_feeder_share = min(1.0, feeder_sold[fid] / feeder_allocated[fid])
                        same_feeder[cid] += allocated_after_self * same_feeder_share
                        other_feeder[cid] += allocated_after_self * (1.0 - same_feeder_share)

            return allocations, need, surplus, same_feeder, other_feeder


In [11]:
# Calculate key performance indicators (KPIs) for a specified energy allocation method over a year
def calculate_kpis_yearly(data, customers, method, dates, dk, calulation):
# Initialize dictionaries (existing + new KPIs)
    detailed_kpis = {
        'leg_revenue': {cid: 0 for cid in customer_ids},
        'no_leg_revenue': {cid: 0 for cid in customer_ids},
        'revenue_increase': {cid: 0 for cid in customer_ids},
        'leg_cost': {cid: 0 for cid in customer_ids},
        'no_leg_cost': {cid: 0 for cid in customer_ids},
        'cost_reduction': {cid: 0 for cid in customer_ids},
        'total_generation': {cid: 0 for cid in customer_ids},
        'total_pv_sold_to_EC': {cid: 0 for cid in customer_ids},
        'total_self_consumption': {cid: 0 for cid in customer_ids},
        'total_pv_sold_grid': {cid: 0 for cid in customer_ids},
        'total_need': {cid: 0 for cid in customer_ids},
        'allocation_from_same_feeder': {cid: 0 for cid in customer_ids},
        'allocation_from_other_feeder': {cid: 0 for cid in customer_ids},
        'total_need_from_grid': {cid: 0 for cid in customer_ids},
        'total_allocation': {cid: 0 for cid in customer_ids},
        'ssr': {cid: 0 for cid in customer_ids},
        # New KPIs

    }
    aggregated_kpis = {
        'leg_revenue': 0,
        'no_leg_revenue': 0,
        'revenue_increase': 0,
        'leg_cost': 0,
        'no_leg_cost': 0,
        'cost_reduction': 0,
        'total_generation': 0,
        'total_pv_sold_to_EC': 0,
        'total_self_consumption': 0,
        'total_pv_sold_grid': 0,
        'total_need': 0,
        'allocation_from_same_feeder': 0,
        'allocation_from_other_feeder': 0,
        'total_need_from_grid': 0,
        'total_allocation': 0,
        'ssr': 0,

    }
    
    # Define pricing constants (unchanged)
    FEED_IN_PRICE = 0.08
    GRID_TOTAL_PRICE = 0.2562
    PV_SELLING_PRICE = 0.1137
    GRID_NETWORK_PRICE = 0.1092
    OTHER_PRICE = 0.0333

  
    # Loop through each timestamp in the date range
    for timestamp in dates:

        pre_need = {cid: data[cid]['EBez'].get(timestamp, 0) for cid in customer_ids}
        total_generation = {cid: data[cid]['EAbg'].get(timestamp, 0) for cid in customer_ids}

        if calulation == 'static':
            allocations, post_need, post_surplus, same_trafo, other_trafo = static_energy_allocation(data, customers, method, timestamp, dk)
        elif calulation == 'dynamic':
            allocations, post_need, post_surplus, same_trafo, other_trafo = dynamic_enegy_allocation(data, customers, method, timestamp, dk)
            

        for cid in customer_ids:
            no_leg_cost = max(0, pre_need[cid] - total_generation[cid]) * GRID_TOTAL_PRICE
            no_leg_revenue = max(0, total_generation[cid] - pre_need[cid]) * FEED_IN_PRICE
            grid_cost = post_need[cid] * GRID_TOTAL_PRICE
            allocated_energy = allocations[cid]
            if allocated_energy > 0:
                same_transformer = same_trafo[cid]
                diff_transformer = other_trafo[cid]
                network_price_same = GRID_NETWORK_PRICE * 0.6
                network_price_diff = GRID_NETWORK_PRICE * 0.8
                pv_cost = (same_transformer * (PV_SELLING_PRICE + network_price_same + OTHER_PRICE) +
                            diff_transformer * (PV_SELLING_PRICE + network_price_diff + OTHER_PRICE))
            else:
                pv_cost = 0
            leg_cost = grid_cost + pv_cost
            grid_revenue = post_surplus[cid] * FEED_IN_PRICE
            pv_sales = (total_generation[cid] - post_surplus[cid] - min(total_generation[cid], pre_need[cid])) * PV_SELLING_PRICE
            leg_revenue = grid_revenue + pv_sales
            
            # Update existing KPIs
            detailed_kpis['leg_revenue'][cid] += leg_revenue
            detailed_kpis['no_leg_revenue'][cid] += no_leg_revenue
            detailed_kpis['revenue_increase'][cid] += leg_revenue - no_leg_revenue
            detailed_kpis['leg_cost'][cid] += leg_cost
            detailed_kpis['no_leg_cost'][cid] += no_leg_cost
            detailed_kpis['cost_reduction'][cid] += no_leg_cost - leg_cost
            detailed_kpis['total_generation'][cid] += total_generation[cid]
            detailed_kpis['total_pv_sold_to_EC'][cid] += total_generation[cid] - post_surplus[cid] - min(total_generation[cid], pre_need[cid])
            detailed_kpis['total_self_consumption'][cid] += min(total_generation[cid], pre_need[cid])
            detailed_kpis['total_pv_sold_grid'][cid] += post_surplus[cid]
            detailed_kpis['total_need'][cid] += pre_need[cid]
            detailed_kpis['allocation_from_same_feeder'][cid] += same_trafo[cid]
            detailed_kpis['allocation_from_other_feeder'][cid] += other_trafo[cid]
            detailed_kpis['total_need_from_grid'][cid] += post_need[cid]
            detailed_kpis['total_allocation'][cid] += allocations[cid]


            # Update aggregated KPIs
            aggregated_kpis['leg_revenue'] += leg_revenue
            aggregated_kpis['no_leg_revenue'] += no_leg_revenue
            aggregated_kpis['revenue_increase'] += leg_revenue - no_leg_revenue
            aggregated_kpis['leg_cost'] += leg_cost
            aggregated_kpis['no_leg_cost'] += no_leg_cost
            aggregated_kpis['cost_reduction'] += no_leg_cost - leg_cost
            aggregated_kpis['total_generation'] += total_generation[cid]
            aggregated_kpis['total_pv_sold_to_EC'] += total_generation[cid] - post_surplus[cid] - min(total_generation[cid], pre_need[cid])
            aggregated_kpis['total_self_consumption'] += min(total_generation[cid], pre_need[cid])
            aggregated_kpis['total_pv_sold_grid'] += post_surplus[cid]
            aggregated_kpis['total_need'] += pre_need[cid]
            aggregated_kpis['allocation_from_same_feeder'] += same_trafo[cid]
            aggregated_kpis['allocation_from_other_feeder'] += other_trafo[cid]
            aggregated_kpis['total_need_from_grid'] += post_need[cid]
            aggregated_kpis['total_allocation'] += allocations[cid]


    # Finalize aggregated KPIs
    aggregated_kpis['ssr'] = aggregated_kpis['total_allocation'] / aggregated_kpis['total_need'] if aggregated_kpis['total_need'] > 0 else 0

    return detailed_kpis, aggregated_kpis